In [8]:
import os
import pandas as pd
import ta   # pip install ta

# === PARAMETERS ===
raw_data_dir       = "./sentimentdataset"
filtered_data_dir  = "./sentimentdataset2"
combined_filename  = "combined_data.csv"

# final date window for combined output
final_start_date   = pd.to_datetime("2021-09-30")
final_end_date     = pd.to_datetime("2022-09-29")

# ensure output dir exists
os.makedirs(filtered_data_dir, exist_ok=True)

# TEMA helper
def calculate_tema(series, window):
    ema1 = ta.trend.EMAIndicator(close=series, window=window).ema_indicator()
    ema2 = ta.trend.EMAIndicator(close=ema1,   window=window).ema_indicator()
    ema3 = ta.trend.EMAIndicator(close=ema2,   window=window).ema_indicator()
    return 3*ema1 - 3*ema2 + ema3

# process one ticker file fully, then slice
def process_csv(file_path, ticker):
    # read & parse dates
    df = pd.read_csv(file_path, parse_dates=["Date"], dayfirst=False)
    df.dropna(subset=["Date"], inplace=True)
    # normalize to naive datetime
    df["Date"] = pd.to_datetime(df["Date"], utc=True).dt.tz_localize(None)

    # DEBUG: show date range
    print(f"{ticker} raw Date range: {df['Date'].min().date()} to {df['Date'].max().date()}")

    # verify necessary columns
    required = ["Open","High","Low","Close","Volume"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        print(f"⚠️ {ticker} missing columns {missing}; skipping.")
        return None

    # keep base columns
    df = df[["Date","Open","High","Low","Close","Volume"]].copy()

    # calculate TA indicators on full range
    for w in [14,26,50,100,200]:
        df[f"SMA_{w}"]  = ta.trend.SMAIndicator(df.Close, window=w).sma_indicator()
        df[f"EMA_{w}"]  = ta.trend.EMAIndicator(df.Close, window=w).ema_indicator()
        df[f"TEMA_{w}"] = calculate_tema(df.Close, w)

    bb = ta.volatility.BollingerBands(df.Close, window=20, window_dev=2)
    df["BB_Hband"]   = bb.bollinger_hband()
    df["BB_Mband"]   = bb.bollinger_mavg()
    df["BB_Lband"]   = bb.bollinger_lband()
    df["RSI_14"]     = ta.momentum.RSIIndicator(df.Close, window=14).rsi()
    macd = ta.trend.MACD(df.Close, window_slow=26, window_fast=12, window_sign=9)
    df["MACD"]       = macd.macd()
    df["MACD_Signal"] = macd.macd_signal()
    df["MACD_Hist"]   = macd.macd_diff()

    # derived & momentum features
    df["Mean_HL"]    = (df.High + df.Low) / 2.0
    df["RMom_14"]    = df.Close / df.Close.shift(14)
    for w in [14,26,50,100,200]:
        df[f"MomTEMA_{w}_ofs1"] = df[f"TEMA_{w}"] / df[f"TEMA_{w}"].shift(1)
        df[f"RCTEMA_{w}"]       = df.Close / df[f"TEMA_{w}"]
        df[f"MomEMA_{w}_ofs1"]  = df[f"EMA_{w}"] / df[f"EMA_{w}"].shift(1)

    df["RTEMA_TEMA_14_50"] = df.TEMA_14 / df.TEMA_50
    df["REMA_EMA_14_50"]   = df.EMA_14  / df.EMA_50
    df["RSMA_SMA_14_50"]   = df.SMA_14  / df.SMA_50
    df["RVolSMA_20"]       = df.Volume / df.Volume.rolling(20).mean()

    # drop warm-up NaNs
    df.dropna(inplace=True)

    # slice final date window
    # slice final date window (by date only)
    df["DateOnly"] = df["Date"].dt.date
    sliced = df[(df["DateOnly"] >= final_start_date.date()) & 
                (df["DateOnly"] <= final_end_date.date())].copy()
    
    if sliced.empty:
        print(f"⚠️ {ticker} has no data in {final_start_date.date()}–{final_end_date.date()}")
        return None
    
    # tag and format
    sliced["Stock Name"] = ticker
    sliced["Date"] = sliced["DateOnly"].astype(str)
    sliced.drop(columns=["DateOnly"], inplace=True)
    return sliced


# main loop
all_frames = []
for fname in os.listdir(raw_data_dir):
    if not fname.lower().endswith('.csv'):
        continue
    ticker = fname.replace('.csv','')
    result = process_csv(os.path.join(raw_data_dir, fname), ticker)
    if result is not None:
        all_frames.append(result)

# combine
if all_frames:
    combined = pd.concat(all_frames, ignore_index=True)
    combined.sort_values(['Stock Name','Date'], inplace=True)
    out_path = os.path.join(filtered_data_dir, combined_filename)
    combined.to_csv(out_path, index=False)
    print(f"✅ Combined CSV saved at {out_path} ({len(combined)} rows)")
else:
    print("⚠️ No data processed; check your files and date range.")


AAPL raw Date range: 1980-12-12 to 2025-01-14
AMD raw Date range: 1980-03-17 to 2025-01-14
AMZN raw Date range: 1997-05-15 to 2025-01-14
BA raw Date range: 1962-01-02 to 2025-01-14
BX raw Date range: 2007-06-22 to 2025-01-14
COST raw Date range: 1986-07-09 to 2025-01-14
CRM raw Date range: 2004-06-23 to 2025-01-14
DIS raw Date range: 1962-01-02 to 2025-01-14
ENPH raw Date range: 2012-03-30 to 2025-01-14
F raw Date range: 1972-06-01 to 2025-01-14
GOOG raw Date range: 2004-08-19 to 2025-01-14
INTC raw Date range: 1980-03-17 to 2025-01-14
KO raw Date range: 1962-01-02 to 2025-01-14
META raw Date range: 2012-05-18 to 2025-01-14
MSFT raw Date range: 1986-03-13 to 2025-01-14
NFLX raw Date range: 2002-05-23 to 2025-01-14
NIO raw Date range: 2018-09-12 to 2025-01-14
NOC raw Date range: 1981-12-31 to 2025-01-14
PG raw Date range: 1962-01-02 to 2025-01-14
PYPL raw Date range: 2015-07-06 to 2025-01-14
TSLA raw Date range: 2010-06-29 to 2025-01-14
TSM raw Date range: 1997-10-09 to 2025-01-14
XPEV 